<a href="https://colab.research.google.com/github/felixchiuman/Stock-Market-Analysis/blob/main/stock_market_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#library

!pip install python-telegram-bot yfinance pandas_ta nest_asyncio mplfinance nltk
from google.colab import userdata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.0/731.0 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 8.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully unin

Bot Code

In [ ]:
import logging
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import nest_asyncio
import asyncio
import os
import time
import datetime
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from telegram import Update
from telegram.constants import ParseMode
from telegram.ext import ApplicationBuilder, ContextTypes, CommandHandler

# --- 1. CONFIGURATION ---
nest_asyncio.apply()
BOT_TOKEN = userdata.get('botToken')

# Download the Brain (VADER Lexicon)
nltk.download('vader_lexicon', quiet=True)
vader = SentimentIntensityAnalyzer()

MEME_URLS = {
    "BUY": "https://i.imgflip.com/1ur9b0.jpg",
    "SELL": "https://i.imgflip.com/261o3j.jpg",
    "WAIT": "https://i.imgflip.com/26am.jpg"
}

# --- 2. NEWS SENTIMENT ENGINE (NEW!) ---
def get_news_sentiment(ticker_symbol):
    """
    Fetches latest news from Yahoo and calculates a Sentiment Score (-1 to 1).
    """
    try:
        stock = yf.Ticker(ticker_symbol)
        news_list = stock.news

        if not news_list:
            return 0.0, "⚪ Neutral (No News)"

        total_score = 0
        count = 0
        headlines = []

        for item in news_list[:5]: # Check last 5 headlines
            title = item.get('title', '')
            score = vader.polarity_scores(title)['compound']
            total_score += score
            count += 1
            headlines.append(f"• {title[:40]}...")

        if count == 0: return 0.0, "⚪ Neutral"

        avg_score = total_score / count

        # Interpret Score
        if avg_score > 0.15:
            sentiment_text = f"🟢 BULLISH ({avg_score:.2f})"
        elif avg_score < -0.15:
            sentiment_text = f"🔴 BEARISH ({avg_score:.2f})"
        else:
            sentiment_text = f"⚪ NEUTRAL ({avg_score:.2f})"

        return avg_score, sentiment_text, headlines

    except Exception as e:
        print(f"News Error: {e}")
        return 0.0, "⚪ Error", []

# --- 3. THE "CHUNKING" DATA FETCHER ---
def get_chunked_data(ticker, years_back=5):
    # (Same robust code as before)
    print(f"🔄 Deep Scan: {ticker} ({years_back} years)...")
    frames = []
    current_year = datetime.date.today().year

    for i in range(years_back):
        year = current_year - i
        start_date = f"{year}-01-01"
        end_date = f"{year}-12-31"
        if year == current_year:
            end_date = datetime.date.today().strftime("%Y-%m-%d")

        try:
            stock = yf.Ticker(ticker)
            df_chunk = stock.history(start=start_date, end=end_date)
            if not df_chunk.empty:
                frames.append(df_chunk)
            time.sleep(0.5)
        except Exception as e:
            pass

    if not frames: return pd.DataFrame()
    full_df = pd.concat(frames).sort_index()
    full_df = full_df[~full_df.index.duplicated(keep='first')]
    return full_df

# --- 4. BACKTEST ENGINE ---
def calculate_backtest_metrics(df):
    wins = 0; losses = 0; gross_profit = 0.0; gross_loss = 0.0; total_trades = 0
    closes = df['Close'].values; means = df['Mean'].values
    std_devs = df['StdDev'].values; z_scores = df['Z_Score'].values
    highs = df['High'].values; lows = df['Low'].values

    for i in range(len(df) - 15):
        if z_scores[i] < -2.0:
            total_trades += 1
            entry_price = closes[i]; target_price = means[i]; stop_price = entry_price - std_devs[i]
            for j in range(1, 11):
                if lows[i+j] <= stop_price:
                    losses += 1; gross_loss += (entry_price - stop_price); break
                elif highs[i+j] >= target_price:
                    wins += 1; gross_profit += (target_price - entry_price); break

    win_rate = (wins / total_trades * 100) if total_trades > 0 else 0
    profit_factor = (gross_profit / gross_loss) if gross_loss > 0 else 0.0
    return win_rate, total_trades, profit_factor

# --- 5. HYBRID ANALYSIS ENGINE ---
def analyze_stock(ticker_input):
    ticker = ticker_input.upper()
    df = get_chunked_data(ticker, years_back=5) # 5 Years

    if df.empty and len(ticker) == 4 and ticker.isalpha():
        ticker = f"{ticker}.JK"
        df = get_chunked_data(ticker, years_back=5)

    if df.empty: return None, None

    # Currency
    currency_symbol = "$"
    try:
        info = yf.Ticker(ticker).info
        if info.get('currency') == 'IDR': currency_symbol = "Rp"
    except: pass

    try:
        # --- A. GET NEWS SENTIMENT ---
        news_score, news_text, headlines = get_news_sentiment(ticker)

        # --- B. MATH INDICATORS ---
        df['Mean'] = df['Close'].rolling(window=20).mean()
        df['StdDev'] = df['Close'].rolling(window=20).std()
        df['Z_Score'] = (df['Close'] - df['Mean']) / df['StdDev']

        real_win_rate, total_trades, real_pf = calculate_backtest_metrics(df)
        years_label = f"{(df.index[-1] - df.index[0]).days / 365:.1f}Y"

        curr = df.iloc[-1]
        price = curr['Close']; z_score = curr['Z_Score']
        std = curr['StdDev']; mean = curr['Mean']

        stop_loss = price - std
        target1 = price + std
        rr_ratio = ((target1 - price) / (price - stop_loss)) if (price-stop_loss) > 0 else 1.0

        # --- C. HYBRID LOGIC (STAT + NEWS) ---

        # ⚠️ THE NEWS FILTER RULE ⚠️
        # If News is very bad (< -0.2) AND Z-Score is Buy, we downgrade the signal.

        swing_action = "💤 WAIT"
        # 1. Check Math First
        if z_score < -2.0:
            if real_win_rate >= 50:
                swing_action = "✅ ENTER (Math Good)"
            else:
                swing_action = "⚠️ RISKY (Math Bad)"
        elif z_score > 2.0:
            swing_action = "🔴 SELL"

        # 2. Apply News Filter
        if "ENTER" in swing_action and news_score < -0.15:
            swing_action = "🛑 BLOCKED (News is Bearish)"

        # Investor Logic
        discount_pct = ((mean - price) / mean) * 100
        invest_action = "💎 BUY" if z_score < -2 else "⚪ HOLD"

        def fmt(val): return f"{int(val):,}" if currency_symbol == "Rp" else f"{val:,.2f}"

        # Headlines String
        headline_txt = "\n".join(headlines[:3])

        report = f"""
📌 **ASSET:** {ticker} (Hybrid AI)
💰 **PRICE:** {currency_symbol} {fmt(price)}
📊 **Z-SCORE:** {z_score:.2f} σ
📰 **NEWS:** {news_text}
──────────────────────
🧠 **HYBRID DECISION**
Math Signal: {z_score:.2f}σ (Oversold)
News Filter: {news_score:.2f} (Sentiment)

👉 **FINAL VERDICT:** {swing_action}
──────────────────────
🐇 **TRADER STATS**
Win Rate: {real_win_rate:.1f}% ({total_trades} Trades)
R:R Ratio: 1:{rr_ratio:.2f}
Stop Loss: {currency_symbol} {fmt(stop_loss)}
──────────────────────
🗞️ **TOP HEADLINES**
{headline_txt}
──────────────────────
Bot: "I read {len(headlines)} headlines. Sentiment is {news_text.split(' ')[1]}."
"""
        meme_key = "WAIT"
        if "ENTER" in swing_action: meme_key = "BUY"
        if "BLOCKED" in swing_action: meme_key = "WAIT"
        if "SELL" in swing_action: meme_key = "SELL"

        return report, MEME_URLS[meme_key]

    except Exception as e:
        print(f"Error: {e}")
        return None, None

# --- 6. HANDLERS ---
async def analyze(update: Update, context: ContextTypes.DEFAULT_TYPE):
    if not context.args: return await update.message.reply_text("Usage: /analyze AAPL")
    ticker = context.args[0]
    await update.message.reply_text(f"🗞️ Reading News & Calculating Math for {ticker}...")

    report, meme = analyze_stock(ticker)
    if report:
        await update.message.reply_photo(meme, caption=report, parse_mode=ParseMode.MARKDOWN)
    else:
        await update.message.reply_text("❌ Data Error.")

async def start(update, context):
    await update.message.reply_text("🤖 Hybrid Bot (News + Math) Ready!")

async def main():
    app = ApplicationBuilder().token(BOT_TOKEN).build()
    app.add_handler(CommandHandler("start", start))
    app.add_handler(CommandHandler("analyze", analyze))
    await app.run_polling()

if __name__ == '__main__':
    try: asyncio.run(main())
    except KeyboardInterrupt: pass